# Central Exercise 6 - Rankine Oval

This script plots streamlines and potential lines for the flow past a Rankine Oval. It results from the superposition of a source- and sink flow with equal strength and a parallel flow. The parallel flow must point in the direction of the line connecting the source and sink.

## Preparation

To prepare the plot, several helper functions have to be defined. This is done in the following cell.
First, the two libraries numpy and matplotlib.pyplot have to be imported in order to user their functionality. Note, these libraries are helpful in many scenarios, when data should be plotted. Next, helper classes that provide the complex velocity for the source-/sink-flow and the parallel flow are implemented. Finally, a function providing the meshgrid for the calculations is implemented.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


class ParallelFlow:

    def __init__(self, velocity_x: float, velocity_y: float) -> None:
        """Initializer for the parallel flow class. Stores relevant quantities as class members.

        Args:
            velocity_x (float): Cartesion component of the velocity pointing in x-direction.
            velocity_y (float): Cartesion component of the velocity pointing in y-direction.
        """
        self.velocity_x = velocity_x
        self.velocity_y = velocity_y

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Return the complex velocity of the parallel flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        return self.velocity_x - 1.0j * self.velocity_y


class SourceFlow:

    def __init__(self, source_strength: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the source- and sink-flow class. Stores relevant quantities as class members.

        Args:
            source_strength (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.source_strength = source_strength
        self.factor = self.source_strength / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the source-/sink-flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z = z - self.translation_x - 1.0j * self.translation_y
        return self.factor / z
    
def get_grid_quantities(x_min: float, x_max: float, y_min: float, y_max: float, resolution_x: int, resolution_y: int):
    """Returns the meshgrid used for the calculations.

    Args:
        x_min (float): The minimum x value.
        x_max (float): The maximum x value.
        y_min (float): The minimum y value.
        y_max (float): The maximum y value.
        resolution_x (int): The number of points to discretize in x direction.
        resolution_y (int): The number of points to discretize in y direction.

    Returns:
        List[np.array]: The numpy array containing the meshgrid.
    """
    x_linspace = np.linspace(x_min, x_max, resolution_x)
    y_linspace = np.linspace(y_min, y_max, resolution_y)
    X,Y = np.meshgrid(x_linspace, y_linspace)
    Z = X + 1.0j * Y
    return X, Y, Z

## Definition of domain

Here, relevant information to describe the domain is provided. It is used to generate the meshgrid for the calculations.

In [ ]:
x_min = -4.0
x_max = 4.0
y_min = -4.0
y_max = 4.0
resolution_per_unit_length = 20
resolution_x = ( x_max - x_min ) * resolution_per_unit_length
resolution_y = ( y_max - y_min ) * resolution_per_unit_length

X, Y, Z = get_grid_quantities(x_min, x_max, y_min, y_max, int(resolution_x), int(resolution_y))

## Calculation of the complex velocity

Initializes the relevant source, sink and parallel flow objects to calculate the complex velocity.

In [ ]:
u_infty = 1.0
y_infty = 1.0
sink_strength = -3.0
source_strength = 3.0

source = SourceFlow(source_strength, -1.0, 0.0)
sink = SourceFlow(sink_strength, 1.0, 0.0)
inflow = ParallelFlow(u_infty, 0.0)

complex_velocity = inflow.get_complex_velocity(Z) + sink.get_complex_velocity(Z) + source.get_complex_velocity(Z)
u = np.real(complex_velocity)
v = - np.imag(complex_velocity)

## Plotting of the results

Plot the streamlines and potential lines.
Streamlines are plotted in TUMBlue.
Potential lines are plotted in TUMOrange.

In [ ]:
fig, ax = plt.subplots()

x_linspace = np.linspace(x_min, x_max, 21)
y_linspace = np.linspace(x_min, x_max, 20)
y_linspace = np.append(y_linspace, 1.e-5)
y_linspace = np.append(y_linspace, -1.e-5)

# Plot the streamlines.

# The streamline seeds describe all the points where streamlines start. Thus, only streamline the go through these points are drawn.
# This may heavily influence the look of the plot!
# In this case only points to the left (*xmin refers to the left, replace with e.g. 0.0 to also obtain streamlines inside the oval)
# of the oval are choosen. Thus, no streamlines inside the oval show.
# In this case, they are equally distributed along the y direction. To also indicate the stagnation streamlines, values slightly above
# and below (+/- 1.e-5) the x axis are added.
streamline_seeds = np.stack([np.ones_like(y_linspace)*x_min, y_linspace], axis=-1)

ax.streamplot(
    X, Y, u, v,
    density=[1.0, 1.0],
    minlength=1.0, maxlength=20.0,
    broken_streamlines=False,
    start_points=streamline_seeds,
    color="#3070b3", # This beautiful color is TUMBlue
)

# Plot the potential lines.

# The potential line seed describe all points where potential lines start. Here, similar remarks as above apply.
potential_line_seeds = np.stack([x_linspace, np.ones_like(x_linspace)*y_min], axis=-1)
ax.streamplot(
    X, Y, -v, u,
    density=[0.2, 0.2],
    minlength=1.0, maxlength=20.0,
    broken_streamlines=False,
    start_points=potential_line_seeds,
    arrowstyle = "-",
    color="#E37222", # This beautiful color is TUMOrange
)

plt.show()